In [ ]:
import sqlite3
from pathlib import Path
 
REPO_PATH = Path("/Users/koen/workspace/toolbox")
SCREENPIPE_PATH = REPO_PATH / "data" / "screenpipe" / "db2.sqlite" 


In [ ]:
HOME = Path.home()

In [ ]:
MEETINGS_DB_PATH = HOME / ".meeting-notes-mcp" / "db.sqlite"
MEETINGS_DB_PATH.parent.mkdir(parents=True, exist_ok=True)

In [ ]:
# MEETINGS_DB_PATH.unlink()

In [ ]:
# Connect to the SQLite database
conn = sqlite3.connect(MEETINGS_DB_PATH)
conn.row_factory = sqlite3.Row

In [ ]:
create_tables(conn)

# Create and query some meetings

In [ ]:
# Connect to the SQLite database
screenpipe_conn = sqlite3.connect(SCREENPIPE_PATH)
screenpipe_conn.row_factory = sqlite3.Row

In [ ]:
for row in screenpipe_conn.execute("""select * from audio_transcriptions""").fetchall():
    insert_transcription(conn, **row)

In [ ]:
meetings = extract_meetings(conn)

In [ ]:
for meeting in meetings:
    print(meeting.filename)
    print(meeting.show_text())
    print()

In [ ]:
for meeting in meetings:
    insert_meeting(conn, meeting.filename, meeting.chunks_ids)

In [ ]:
for row in get_all_meeting_notes(conn):
    print(dict(row))

table meeting_meta

meeting_id | filename

table meeting_chunks

meeting_i | chunkid

```


```


Meetings CTE
| Meeting id | filename | full text | start date | end date


```
WITH meetings AS (
  SELECT
    mc.meeting_id,
    mm.filename,
    MIN(mc.date) AS start_date,
    MAX(mc.date) AS end_date,
    GROUP_CONCAT(mc.text, ' ') AS full_text
  FROM (
    SELECT * FROM meeting_chunks ORDER BY date, chunkid
  ) mc
  JOIN meeting_meta mm ON mc.meeting_id = mm.meeting_id
  GROUP BY mc.meeting_id
)
SELECT *
FROM meetings;
```

In [ ]:
for chunks in meetings:
    date = chunks[0]["timestamp"]
    print(f"meeting {date}")
    for chunk in chunks:
        print(chunk["audio_chunk_id"])
    
    print()
    